In [39]:
#---------------------------------------------------------------------------------------------LIBRARIES--------------------------------------------------------------------------------------------
import cv2                                                                                             #import OpenCV2 library for image processing and algorithms
import math
import numpy as np                                                                                     #import numpy mathematical library
import operator                                                                                        #additional efficient pyhton fucntions 
import matplotlib.pyplot as plt                                                                        #import matplotlib library for plotting
from scipy import ndimage                                                                              #package contains various functions for multidimensional image processing                       
from webcolors import rgb_to_name, name_to_rgb                                                         #import the webcolors library which enables RGB to name and vice versa conversions
from IPython.core.display import display, HTML                                    
display(HTML("<style>.container { width:100% !important; }</style>"))                                  #change width of Jupyer Notebook to use the whole window resolution available

In [ ]:
###### --------------------------------------------------------------------------------------------DEFINITIONS-------------------------------------------------------------------------------------------
def map_value(value, in_low, in_high, out_low, out_high):                                              #create Arduino map() function in python for usage throughout the code
    return out_low + (out_high - out_low) * ((value - in_low) / (in_high - in_low))                    #scale input lowest,input highest range to output lowest,output highest range then return


#----------------------------------------------------------------------------------------IMAGE PRE-PROCESSING---------------------------------------------------------------------------------------
colour_img = cv2.imread('./uno_images/b9.jpg')                                                         #load the image from the specified path
rotated_img = ndimage.rotate(colour_img, -90)                                                          #rotate image by 90 degrees, increase user ease of use
rotated_img_copy = rotated_img.copy()                                                                  #create a copy of the rotated_img so that they don't share the same memory address
bw_img = cv2.cvtColor(rotated_img, cv2.COLOR_BGR2GRAY)                                                 #convert to a black and white image


#-------------------------------------------------------------------------------------------POST-PROCESSING-----------------------------------------------------------------------------------------
thr_value, th_img = cv2.threshold(bw_img,150, 400, cv2.THRESH_BINARY_INV)                              #accurate countours, smoother edges compared to regular binary
#very small changes with images provided, helps with countour accuracy
kernel = np.ones((3, 3), np.uint8)                                                                     #higher kernel = less accurate contours
#close_img = cv2.morphologyEx(img_th, cv2.MORPH_CLOSE, kernel)                                         #erosion + dilute method (internal spaces removal)
open_img = cv2.morphologyEx(th_img, cv2.MORPH_OPEN, kernel)                                            #dilute + erosion method (noise removal)


#-----------------------------------------------------------------------------------EDGE DETENCTION & CONTOUR MAPPING---------------------------------------------------------------------------------
canny_img = cv2.Canny(open_img, 50, 100)                                                               #edge detection using the OpenCV Canny method
contours, _ = cv2.findContours(open_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)  
selected_contour = 1                                                                                   #0 = frame; 1 = uno card perimeter;
contours_img = cv2.drawContours(rotated_img, contours, selected_contour, (0,255,0), 2, cv2.LINE_AA)    #draw selected contour on top of the rotated colour image
#print(len(contours))                                                                                  #debugging: show how many contours have been found
#print(hierarchy)                                                                                      #debugging: show hierarchy list for all contours



#---------------------------------------------------------------------------OPEN CV CONTOUR CROPPING USING CONTOURS FUNCTION--------------------------------------------------------------------------- 
x_cnt,y_cnt,w_cnt,h_cnt = cv2.boundingRect(contours[selected_contour])                                 #find origin, width and heigth of image based on selected_contour
contour_cropped_proc_img = rotated_img[y_cnt:y_cnt+h_cnt, x_cnt:x_cnt+w_cnt]                           #crop the image based on the coordinates found for processing
framed_img = cv2.copyMakeBorder(contour_cropped_proc_img,                                              #add a 30px wide black frame around the cropped image (helps with text placement) 
                                30,30,30,30,
                                cv2.BORDER_CONSTANT,
                                value=(0,0,0)) 
#print(x_cnt, y_cnt, w_cnt, h_cnt)                                                                     #print 4 values for debugging
 

#-------------------------------------------------------------------------------------------COLOUR ANALYSIS--------------------------------------------------------------------------------------------
contour_cropped_analysis_img = rotated_img_copy[y_cnt:y_cnt+h_cnt, x_cnt:x_cnt+w_cnt]                  #crop the rotated original image copy with the uno card contour for pattern/colour processing
#a patch of pixels is chosen to increase accuracy 
#as well as optimise the code by running through less pixels in the functions to follow
colour_patch_img = contour_cropped_analysis_img[40:80, 100:140]                                        #select a constant patch from the uno card image to perform the pixel analysis
bgr_pixels = colour_patch_img.tolist()                                                                 #transform patch array to list containing bgr tuples
#print(bgr_pixels)
b = [x[0][0] for x in bgr_pixels]                                                                      #extract blue values from bgr list
g = [x[0][1] for x in bgr_pixels]                                                                      #extract green values from bgr list
r = [x[0][2] for x in bgr_pixels]                                                                      #extract red values from bgr list
frequent_b = (max(set(b), key = b.count))                                                              #find the most frequent blue value in the image patch selected
frequent_g = (max(set(g), key = g.count))                                                              #find the most frequent green value in the image patch selected
frequent_r = (max(set(r), key = r.count))                                                              #find the most frequent red value in the image patch selected


#-----------------------------------------------------------------------------------------COLOUR IDENTIFICATION------------------------------------------------------------------------------------------    
rgb_dictionary = {"red": frequent_r, "green": frequent_g, "blue": frequent_b}                          #create dictionary containing the RGB colour space, and assign most frequent value from patch
sorted_rgb_dictionary = dict((y, x) for y, x in sorted(rgb_dictionary.items(),                         #sort dictionary based on value, not key and since the output is a tuple, transform to dictionary
                                                       key=operator.itemgetter(1)))                    #choose value for sorting process, (1) = value, (0) = key
highest_rgb_value = list(sorted_rgb_dictionary)[2]                                                     #extract highest value whether it is a blue, green or red pixel (key)
middle_rgb_value = list(sorted_rgb_dictionary)[1]                                                      #extract middle value whether it is a blue, green or red pixel (key)
lowest_rgb_value = list(sorted_rgb_dictionary)[0]                                                      #extract lowest value whether it is a blue, green or red pixel (key)

#the following part of the code changes the R,G,B values of the pixels to an extreme
#this helps identify the colour correctly with different levels of brightness in the image
colour_rgb = [0,0,0]                                                                                   #create list to store the new RGB values for colour identification
if sorted_rgb_dictionary.get(highest_rgb_value) >= 255/2:                                              #if the highest pixel value is higher than the mid-point, then:
    if highest_rgb_value == "red":                                                                     #if the highest_rgb_value is a "red" pixel, then:
        colour_rgb[0] = 255                                                                                #assign the first element in the colour_rgb = 255 
    elif highest_rgb_value == "green":                                                                 #if the highest_rgb_value is a "green" pixel, then:
        colour_rgb[1] = 255                                                                                #assign the second element in the colour_rgb = 255 
    elif highest_rgb_value == "blue":                                                                  #if the highest_rgb_value is a "blue" pixel, then:
        colour_rgb[2] = 255                                                                                #assign the third element in the colour_rgb = 255 
elif sorted_rgb_dictionary.get(highest_rgb_value) <= 255/2:                                            #if the highest pixel value is lower than the mid-point, then:
    if highest_rgb_value == "red":                                                                     #if the highest_rgb_value is a "red" pixel, then:
        colour_rgb[0] = 128                                                                                #assign the first element in the colour_rgb = 128 
    elif highest_rgb_value == "green":                                                                 #if the highest_rgb_value is a "green" pixel, then:
        colour_rgb[1] = 128                                                                                #assign the second element in the colour_rgb = 128 
    elif highest_rgb_value == "blue":                                                                  #if the highest_rgb_value is a "blue" pixel, then:
        colour_rgb[2] = 128                                                                                #assign the third element in the colour_rgb = 128 
if sorted_rgb_dictionary.get(middle_rgb_value) > 150:                                                  #if the middle pixel value is higher than 150, then:
    if middle_rgb_value == "red":                                                                      #if the middle_rgb_value is a "red" pixel, then:
        colour_rgb[0] = 255                                                                                #assign the first element in the colour_rgb = 255 
    elif middle_rgb_value == "green":                                                                  #if the middle_rgb_value is a "green" pixel, then:
        colour_rgb[1] = 255                                                                                #assign the second element in the colour_rgb = 255 
    elif middle_rgb_value == "blue":                                                                   #if the middle_rgb_value is a "blue" pixel, then:
        colour_rgb[2] = 255                                                                                #assign the third element in the colour_rgb = 255 
if sorted_rgb_dictionary.get(lowest_rgb_value) < 255/2:                                                #if the lowest pixel value is lower than the mid-point, then:
    if lowest_rgb_value == "red":                                                                      #if the lowest_rgb_value is a "red" pixel, then:
        colour_rgb[0] = 0                                                                                  #assign the first element in the colour_rgb = 0 
    elif lowest_rgb_value == "green":                                                                  #if the lowest_rgb_value is a "green" pixel, then:
        colour_rgb[1] = 0                                                                                  #assign the second element in the colour_rgb = 0 
    elif lowest_rgb_value == "blue":                                                                   #if the lowest_rgb_value is a "blue" pixel, then:
        colour_rgb[2] = 0                                                                                  #assign the third element in the colour_rgb = 0 
        
named_colour = rgb_to_name(colour_rgb)                                                                 #use webcolours library database to convert RGB to HEX and then to colour name in English 
colour_framed_img = cv2.copyMakeBorder(framed_img,7,7,7,7,cv2.BORDER_CONSTANT,                         #add a 7px wide frame at the edge of the frame with the same colour as the UNO card
                                       value=(colour_rgb[2],colour_rgb[1],colour_rgb[0]))              #elements 2,1,0 used because of BGR output, and RGB input
"""
#Debugging
test = name_to_rgb('cyan') 
print(test)
print(sorted_rgb_dictionary)
print(type(highest_rgb_value))
print(lowest_rgb_value, middle_rgb_value, highest_rgb_value)
print(colour, named_colour)
"""


#-------------------------------------------------------------------------------------RESOLUTION NORMALIZATION------------------------------------------------------------------------------------------
contour_cropped_copy = contour_cropped_analysis_img.copy()                                             #create a copy of the uno card contour cropped image
normalize_resolution = (300, 450)                                                                      #have a set resolution between all images (after cropping)
normalized_img = cv2.resize(contour_cropped_copy, normalize_resolution,                                #set image to be resized and the resolution
                                        interpolation = cv2.INTER_AREA)

cv2.imshow('normalized', normalized_img)


#------------------------------------------------------------------------------------SHAPE IMAGE POST-PROCESSING---------------------------------------------------------------------------------------
normalized_img_bw = cv2.cvtColor(normalized_img, cv2.COLOR_BGR2GRAY)                                       #convert rescaled center image to black and white channels for post-processing
thr_value2, th_shape_img = cv2.threshold(normalized_img_bw, 0, 250,cv2.THRESH_OTSU)              #accurate countours, smoother edges compared to regular binary
kernel_close = np.ones((3, 3), np.uint8)                                                               #higher kernel = less accurate contours
morph_shape_img = cv2.morphologyEx(th_shape_img , cv2.MORPH_CLOSE, kernel_close)                        #erosion + dilute method (internal spaces removal)
canny_shape_img = cv2.Canny(morph_shape_img, 150, 500)  



contours_shape, hierarchy_shape = cv2.findContours(canny_shape_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE) 
print(type(contours_shape), len(contours_shape))
for i, c in enumerate(contours_shape):         # loop through all the found contours
    perimeter = cv2.arcLength(c, True)     # perimeter of contour c (curved length)     
    area = cv2.contourArea(c) 
    polygon_constant = 0.04    # try changing this value to get more or less corners
    corners = len(cv2.approxPolyDP(c, polygon_constant*perimeter, True)) 
    ellipse = cv2.fitEllipse(c)    # fit an ellipse on the contour
    (center, axes, orientation) = ellipse   # extract the main parameter
    majoraxis_length = max(axes)
    minoraxis_length = min(axes)
    #print(area)
    if not 4 < corners < 15 or 15 < area/perimeter < 25 or 1.4 < (majoraxis_length/minoraxis_length) < 1.6:
        contours_shape.pop(i) 
    print(type(contours_shape), len(contours_shape))



shape_selectedcountour = 7
drawn_shape_contours_img = cv2.drawContours(normalized_img, contours_shape, shape_selectedcountour, (0,255,0), 2, cv2.LINE_AA)
cv2.imshow('drawn_shape_contours_img', drawn_shape_contours_img)
cv2.imshow('morph_shape_img', morph_shape_img) 

mask = np.zeros_like(morph_shape_img) # Create mask where white is what we want, black otherwise
cv2.drawContours(mask, contours_shape, shape_selectedcountour, 255, -1) # Draw filled contour in mask
center_masked = np.zeros_like(morph_shape_img) # Extract out the object and place into output image
center_masked[mask == 255] = morph_shape_img[mask == 255]
cv2.imshow('center_masked', center_masked)

#---------------------------------------------------------------------------------------FEATURE SPACE RECOGNITION---------------------------------------------------------------------------------------
perimeter = cv2.arcLength(contours_shape[shape_selectedcountour], True)
area = cv2.contourArea(contours_shape[shape_selectedcountour])
polygon_constant = 0.04    # try changing this value to get more or less corners
corners = len(cv2.approxPolyDP(contours_shape[shape_selectedcountour], polygon_constant*perimeter, True)) 
print(area/perimeter, corners)


ellipse = cv2.fitEllipse(contours_shape[shape_selectedcountour])    # fit an ellipse on the contour
(center, axes, orientation) = ellipse   # extract the main parameter
majoraxis_length = max(axes)
minoraxis_length = min(axes)

print("Tallness: ", majoraxis_length/minoraxis_length)



harris_method_RGB_img= cv2.cvtColor(center_masked.copy(),cv2.COLOR_GRAY2RGB)
harris_method_bw_img = np.float32(cv2.cvtColor(harris_method_RGB_img, cv2.COLOR_BGR2GRAY))
dst = cv2.cornerHarris(harris_method_bw_img,2,3,0.04)
#result is dilated for marking the corners, not important
dst = cv2.dilate(dst,None)
# Threshold for an optimal value, it may vary depending on the image.
harris_method_RGB_img[dst>0.01*dst.max()]=[0,0,255]
cv2.imshow('harris',harris_method_RGB_img)

thresh = math.exp(19)
harris_corners = 0
for i in range(dst.shape[0]):
    for j in range(dst.shape[1]):
        if int(dst[i,j]) > thresh:
            harris_corners = harris_corners + 1
print("Harris corners: ", harris_corners)


shi_tomasi_RGB_img= cv2.cvtColor(center_masked.copy(),cv2.COLOR_GRAY2RGB)
shi_tomasi_method_bw_img = cv2.cvtColor(shi_tomasi_RGB_img, cv2.COLOR_BGR2GRAY)
corners = cv2.goodFeaturesToTrack(shi_tomasi_method_bw_img, 0, 0.25, 0.05)
  
corners = np.int0(corners)
print("Shi-Tomasi Corners Detected: ", corners)
# draw red color circles on all corners
for i in corners:
    x, y = i.ravel()
    cv2.circle(shi_tomasi_RGB_img, (x, y), 3, (0, 0, 255), -1)

cv2.imshow('shi_tomasi',shi_tomasi_RGB_img)



houghcircles_RGB_img= contour_cropped_analysis_img.copy()
houghcircles_bw_img = cv2.cvtColor(houghcircles_RGB_img, cv2.COLOR_BGR2GRAY)

rows = houghcircles_bw_img.shape[0]
circles = cv2.HoughCircles(houghcircles_bw_img, cv2.HOUGH_GRADIENT, 1, rows / 16,
                           param1=100, param2=30,
                           minRadius=34, maxRadius=35)
print(type(circles), circles)

if circles is not None:
    circles = np.uint16(np.around(circles))
    for i in circles[0, :]:
        center = (i[0], i[1])
        # circle center
        cv2.circle(houghcircles_RGB_img, center, 1, (0, 100, 100), 3)
        # circle outline
        radius = i[2]
        cv2.circle(houghcircles_RGB_img, center, radius, (255, 0, 255), 3)

cv2.imshow('houghcircles_RGB_img',houghcircles_RGB_img)

#------------------------------------------------------------------------------------IDENTIFICATION TEXT PLACEMENT--------------------------------------------------------------------------------------                                                                      
disp_x = +240                                                                                          #perfect value = +240
disp_y = -165                                                                                          #perfect value = -165
text_displayed = (named_colour + " " + " uno card").upper()
identified_img = cv2.putText(colour_framed_img,                                                        #create text on top of image
                             text_displayed,                                                           #set text to text_displayed string
                             (y_cnt+disp_y, x_cnt+disp_x),                                             #set coords to contour bottom right corner with displacement
                             cv2.FONT_HERSHEY_SIMPLEX,                                                 #set OpenCV font
                             0.4, (0,255,0), 1, cv2.LINE_AA)                                           #font size, colour, thickness, antialiasing on the text for smoother edges


#------------------------------------------------------------------------------------POST-PROCESSED IMAGE SCALLING-------------------------------------------------------------------------------------                                                                                             
scaling = int(identified_img.shape[1]*1.5),int(identified_img.shape[0]*1.5)                            #scale the output image by 1.5x
final_img = cv2.resize(identified_img, scaling, interpolation = cv2.INTER_AREA)                        #scale the identified_img, with scaling resolution


#-------------------------------------------------------------------------------------------IMAGE DISPLAY---------------------------------------------------------------------------------------------- 
cv2.imshow('original', colour_img)                                                                     #show the original image to the user
#cv2.imshow('cropped', contour_cropped_analysis_img)                                                    #show the contour cropped image to the user
cv2.imshow('final_img', final_img)                                                                     #show the final image to the user
  
cv2.imshow('canny_shape_img', canny_shape_img)   



key = cv2.waitKey(0)                                                                                   #wait for any key press
cv2.destroyAllWindows()                                                                                #close all windows displaying images

<class 'list'> 22
<class 'list'> 21
<class 'list'> 20
<class 'list'> 20
<class 'list'> 20
<class 'list'> 20
<class 'list'> 20
<class 'list'> 19
<class 'list'> 18
<class 'list'> 17
<class 'list'> 17
<class 'list'> 17
<class 'list'> 16
<class 'list'> 16
<class 'list'> 16
<class 'list'> 16
<class 'list'> 16
6.615596896100087 4
Tallness:  6.899001018227238
Harris corners:  12
Shi-Tomasi Corners Detected:  [[[108 298]]

 [[190 299]]

 [[153 299]]

 [[135 299]]

 [[106 296]]

 [[190 286]]

 [[107 286]]]
<class 'NoneType'> None
